In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt

In [2]:
df = pd.read_parquet("/h/dsmith/physionet_data/prepped/physionet_data.parquet", engine="pyarrow")
npts = 10
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,SepsisEver,id
0,71.0,93.0,36.56,104.0,68.0,51.0,6.0,33.017979,0.000000,26.0,...,334.000,71.23,1.0,1.0,0.0,-70.52,1.0,0.0,0.0,17001.0
1,71.0,93.0,36.56,104.0,68.0,51.0,6.0,33.017979,0.000000,26.0,...,334.000,71.23,1.0,1.0,0.0,-70.52,2.0,0.0,0.0,17001.0
2,73.0,98.5,36.56,103.0,67.5,50.0,13.5,33.017979,0.000000,26.0,...,334.000,71.23,1.0,1.0,0.0,-70.52,3.0,0.0,0.0,17001.0
3,74.0,99.0,36.56,116.0,73.0,53.0,13.5,33.017979,0.000000,26.0,...,334.000,71.23,1.0,1.0,0.0,-70.52,4.0,0.0,0.0,17001.0
4,82.0,98.0,36.56,109.0,72.5,54.5,18.5,33.017979,0.111111,26.0,...,334.000,71.23,1.0,1.0,0.0,-70.52,5.0,0.0,0.0,17001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545247,83.0,98.0,37.60,109.0,61.0,41.0,17.0,33.017979,-2.000000,22.0,...,116.250,73.02,1.0,0.0,1.0,-84.92,20.0,0.0,0.0,11000.0
1545248,80.0,98.0,37.67,112.0,59.0,40.0,20.0,33.017979,-2.000000,22.0,...,118.625,73.02,1.0,0.0,1.0,-84.92,21.0,0.0,0.0,11000.0
1545249,82.0,98.0,37.67,108.0,56.0,37.0,15.0,33.017979,-2.000000,22.0,...,121.000,73.02,1.0,0.0,1.0,-84.92,22.0,0.0,0.0,11000.0
1545250,86.0,97.0,37.67,129.0,58.0,45.0,18.0,33.017979,-2.000000,22.0,...,121.000,73.02,1.0,0.0,1.0,-84.92,23.0,0.0,0.0,11000.0


In [3]:
badcols = ["ICULOS", "SepsisLabel"]
slim = df[[i for i in df.columns if i not in badcols]]
variations = slim.groupby("id").std().mean()
static_vars = variations[variations == 0].index.tolist()
puny_vars = variations[(0 < variations) & (variations < 0.05)].index.tolist()
moving_vars = [i for i in variations.index.tolist() if i not in puny_vars + static_vars]

In [4]:
data_groups = {
    "labels": slim.groupby("id")["SepsisEver"].tail(1),
    "skew": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").skew().reset_index(),
    "std": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").std().reset_index(),
    "mean": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").mean().reset_index(),
    "static": slim[static_vars + ["id"]].groupby("id").tail(1).reset_index(),
}

In [5]:
var_sets = {
    "kitchen_sink": ["std", "mean", "static", "skew"],
    "static": ["static"],
    "moving": ["mean", "std", "kurt", "skew"],
    "tail": ["mean"]
}

In [6]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_validate

class BoostModel(object):
    
    def __init__(self, X, y, group="none", nparts=10):
        self.Xdata = X
        self.ydata = y
        self.nparts = nparts
        self.results = pd.DataFrame()
        self.dgroup = group
        
    def optimize(self):
        depths = [2, 3, 5, 7, 10]
        min_leafs = [2, 8, 32, 128, 256]
        
        for i in depths:
            for j in min_leafs:
                model = RFC(max_depth=i, min_samples_leaf=j, class_weight="balanced")
                cv_results = cross_validate(model, self.Xdata, self.ydata, cv=self.nparts, scoring=("f1", "accuracy"))
                self.results = self.results.append(
                    {
                        "depth": i,
                        "min_leafs": j,
                        "data_group": self.dgroup,
                        "test_acc": cv_results["test_accuracy"].mean(),
                        "test_acc_std": cv_results["test_accuracy"].std(),
                        "test_f1": cv_results["test_accuracy"].mean(),
                        "test_f1_std": cv_results["test_accuracy"].std(),
                    },
                    ignore_index=True
                )
                print(
                    "Random Forest with depth {} and min_leafs {} got accuracy = {} +/- {} on {}.".format(
                        i, j, cv_results["test_accuracy"].mean(), cv_results["test_accuracy"].std(), self.dgroup
                    )
                )
        
        
def assemble_data(var_set):
    chosen_sets = var_sets[var_set]
    
    dataset = data_groups[chosen_sets[0]].copy()
    for i in range(1, len(chosen_sets)):
        dataset = dataset.merge(data_groups[chosen_sets[i]], on="id")
    
    del dataset["id"]
    return dataset

 
all_results = pd.DataFrame()
for i in var_sets:
    print("\nPreparing {}...".format(i))
    xdata = assemble_data(i)
    print("Data prepped. Running models.")
    BM = BoostModel(xdata, data_groups["labels"], group=i)
    BM.optimize()
    all_results = all_results.append(BM.dgroup)
    
all_results.to_csv("model_opt_results.csv", index=False)


Preparing kitchen_sink...
Data prepped. Running models.


/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/ssd003/projects/aieng/public/forec

Random Forest with depth 2 and min_leafs 2 got accuracy = nan +/- nan on kitchen_sink.


/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/ssd003/projects/aieng/public/forec

Random Forest with depth 2 and min_leafs 8 got accuracy = nan +/- nan on kitchen_sink.


...

/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/ssd003/projects/aieng/public/forec

Random Forest with depth 10 and min_leafs 128 got accuracy = nan +/- nan on kitchen_sink.


/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/ssd003/projects/aieng/public/forecasting_unified/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/ssd003/projects/aieng/public/forec

Random Forest with depth 10 and min_leafs 256 got accuracy = nan +/- nan on kitchen_sink.


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid